存储多维数据时：Pandas 提供了 Panel 和 Panel4D 对象解决三维数据与四维数据。

在实践中，是通过层级索引（也被称为多级索引）配合多个有不同等级的一级索引一起使用，可以将高维数组转换成类似一维 Series 和二维 DataFrame 对象的形式。

本节介绍创建 MultiIndex 对象的方法，多级索引数据的取值、切片和统计值的计算，以及普通索引与层级索引的转换方法。

In [1]:
import pandas as pd
import numpy as np

# 1-多级索引Series
    用一维的 Series 对象表示二维数据

01--笨方法

In [2]:
# 分析美国各州在两个不同年份的数据

# 用一个 Python 元组来表示索引
index = [('California', 2000),
         ('California', 2010),
         ('New York', 2000),
         ('New York', 2010),
         ('Texas', 2000), 
         ('Texas', 2010)]
populations = [33871648, 37253956, 18976457, 19378102, 20851820, 25145561]

pop = pd.Series(populations,index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

In [3]:
# 通过元组构成的多级索引，可以直接在 Series 上取值或用切片查询数据
pop[('California', 2010):('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

In [4]:
# 直接取值
pop[('Texas', 2000)]

20851820

In [5]:
# 选择所有 2000 年的数据
pop[[i for i in pop.index 
         if i[1] == 2010]]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

02--好办法：Pandas多级索引
    
    Pandas 的 MultiIndex 类型提供了更丰富的操作方法

In [6]:
# 用元组创建一个多级索引（ index 是一个元组）
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex([('California', 2000),
            ('California', 2010),
            (  'New York', 2000),
            (  'New York', 2010),
            (     'Texas', 2000),
            (     'Texas', 2010)],
           )

In [7]:
# 将前面创建的 pop 的索引重置为 MultiIndex，就会看到层级索引
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

前两列表示 Series 的多级索引值，第三列是数据

In [8]:
# 可直接用第二个索引获取 2010 年的全部数据，与 Pandas 的切片查询用法一致
pop[:, 2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

03--高维数据的多级索引

可以用一个带行列索引的简单 DataFrame 代替前面的多级索引

    unstack() 方法可以快速将一个多级索引的 Series 转化为普通索引的 DataFrame

In [9]:
# 将带有一个多级索引的 Series 转换为 DataFrame 
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [10]:
# stack() 方法实现相反的效果（将 DataFrame 转换为带有一个多级索引的 Series）
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

为什么要费时间研究层级索引?

    理由很简单：用含多级索引的一维 Series 数据表示二维数据，那么就可以用 Series 或 DataFrame 表示三维甚至更高维度的数据。
    多级索引每增加一级，就表示数据增加一维，利用这一特点就可以表示任意维度的数据了。

In [11]:
# 增加一列显示每一年各州的人口统计指标
pop_df = pd.DataFrame({'total':pop,
                       'under18':[9267089, 9284094, 4687374,
                                  4318033, 5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

In [12]:
# 多级索引形式
pop_df.stack()

California  2000  total      33871648
                  under18     9267089
            2010  total      37253956
                  under18     9284094
New York    2000  total      18976457
                  under18     4687374
            2010  total      19378102
                  under18     4318033
Texas       2000  total      20851820
                  under18     5906301
            2010  total      25145561
                  under18     6879014
dtype: int64

In [13]:
# 通用函数和其他功能也同样适用于层级索引
f_u18 = pop_df['under18'] / pop_df['total']
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


# 2-多级索引的创建方法
    为 Series 或 DataFrame 创建多级索引最直接的办法就是将 index 参数设置为至少二维的索引数组。

In [14]:
df = pd.DataFrame(np.random.rand(4,2),
                  index=[['a','a','b','b'],[1,2,1,2]],columns=['data1','data2'])
df

data1     data2
a 1  0.385266  0.376191
  2  0.334002  0.285177
b 1  0.828237  0.583159
  2  0.067065  0.873472

In [15]:
# 如果把将元组作为键的字典传递给 Pandas，Pandas 也会默认转换为 MultiIndex
data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}
pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

有时候显式地创建 MultiIndex 也是很有用的

01--显式地创建多级索引
    
    用 pd.MultiIndex 中的类方法更加灵活地构建多级索引
    在创建 Series 或 DataFrame 时，可以将这些对象作为 index 参数，
    或者通过 reindex 方法更新 Series 或 DataFrame 的索引。

In [16]:
# 通过一个有不同等级的若干简单数组组成的列表来构建 MultiIndex
pd.MultiIndex.from_arrays([['a','a','b','b'],[1,2,1,2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [17]:
# 通过包含多个索引值的元组构成的列表创建 MultiIndex
pd.MultiIndex.from_tuples([('a',1),('a',2),('b',1),('b',2)])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [18]:
# 两个索引的笛卡尔积（Cartesian product）创建 MultiIndex
pd.MultiIndex.from_product([['a','b'],[1,2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [19]:
# 更可以直接提供 levels（包含每个等级的索引值列表的列表）和
# labels（包含每个索引值标签列表的列表）创建 MultiIndex
# labels 标签已被抛弃，用 code 标签代替 
pd.MultiIndex(levels=[['a','b'],[1,2]],codes=[[0,0,1,1],[0,1,0,1]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

02--多级索引的等级名称
    
    给 MultiIndex 的等级加上名称会为一些操作提供便利
    可在 MultiIndex 构造器中通过 names 参数设置等级名称，
    也可以在创建之后通过索引的 names 属性来修改名称。

In [20]:
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

03--多级列索引
    
    每个 DataFrame 的行与列都是对称的，
    也就是说既然有多级行索 引，那么同样可以有多级列索引

In [21]:
# 医学报告的模拟数据，创建了一个简易的四维数据
# 多级行列索引
index = pd.MultiIndex.from_product([[2013,2014],[1,2]],names=['year','visit'])
columns = pd.MultiIndex.from_product([['Bob','Guido','Sue'],['HR','Temp']],
                                     names=['subject','type'])

# 模拟数据
data = np.round(np.random.randn(4,6),1)
data[:,::2] *= 10
data += 37

# 创建 DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      14.0  38.2  39.0  36.1  53.0  37.9
     2      46.0  34.5  36.0  38.8  47.0  38.1
2014 1      44.0  37.0  25.0  38.0  67.0  35.9
     2      44.0  37.7  26.0  37.7  52.0  36.6

In [22]:
health_data.stack()

subject           Bob  Guido   Sue
year visit type                   
2013 1     HR    14.0   39.0  53.0
           Temp  38.2   36.1  37.9
     2     HR    46.0   36.0  47.0
           Temp  34.5   38.8  38.1
2014 1     HR    44.0   25.0  67.0
           Temp  37.0   38.0  35.9
     2     HR    44.0   26.0  52.0
           Temp  37.7   37.7  36.6

In [23]:
# 可以在列索引的第一级查询姓名，
# 从而获取包含一个人（例如 Guido）全部检查信息的 DataFrame
health_data['Guido']

type          HR  Temp
year visit            
2013 1      39.0  36.1
     2      36.0  38.8
2014 1      25.0  38.0
     2      26.0  37.7

In [24]:
health_data['Guido'].unstack()

type     HR        Temp      
visit     1     2     1     2
year                         
2013   39.0  36.0  36.1  38.8
2014   25.0  26.0  38.0  37.7

# 3-多级索引的取值与切片
    把索引看成额外增加的维度

01--Series 多级索引

In [25]:
# 由各州历年人口数量创建的多级索引 Series
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [26]:
# 通过对多个级别索引值获取单个元素
pop['California',2000]

33871648

In [27]:
# MultiIndex 也支持局部取值，只取索引的某 一个层级。
# 只取最高级的索引，获得的结果是一个新的 Series，未被选中的低层索引值会被保留
pop['California']

year
2000    33871648
2010    37253956
dtype: int64

In [28]:
# 局部切片，要求 MultiIndex 是按顺序排列的
pop.loc['California':'New York']

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

In [29]:
# 若索引已经排序，那么可以用较低层级的索引取值，第一层级的索引可以用空切片。
pop[:, 2000]

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

In [30]:
# 通过布尔掩码选择数据
pop[pop > 22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

In [31]:
# 花哨的索引选择数据
pop[['California','Texas']]

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

02-DataFrame 多级索引
    
    DataFrame 多级索引的用法与 Series 类似。

In [32]:
# 体检报告数据
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      14.0  38.2  39.0  36.1  53.0  37.9
     2      46.0  34.5  36.0  38.8  47.0  38.1
2014 1      44.0  37.0  25.0  38.0  67.0  35.9
     2      44.0  37.7  26.0  37.7  52.0  36.6

由于 DataFrame 的基本索引是列索引，因此 Series 中多级索引的用法到了 DataFrame 中就应用在列上了。

In [33]:
# 获取 Guido 的心率数据
health_data['Guido','HR']

year  visit
2013  1        39.0
      2        36.0
2014  1        25.0
      2        26.0
Name: (Guido, HR), dtype: float64

In [34]:
# loc、iloc 索引器都可以使用
health_data.iloc[:2,:2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      14.0  38.2
     2      46.0  34.5

In [35]:
# 在 loc 和 iloc 中可以传递多个层级的索引元组
health_data.loc[:,('Bob', 'HR')]

year  visit
2013  1        14.0
      2        46.0
2014  1        44.0
      2        44.0
Name: (Bob, HR), dtype: float64

In [36]:
# 索引元组的用法不是很方便，如果在元组中使用切片还会导致语法错误
health_data.loc[(:,1),(:,'HR')]

SyntaxError: invalid syntax (<ipython-input-36-7e85074decaa>, line 2)

In [37]:
# 用 Python 内置的 slice() 函数获取想要的切片
# 还有一种更好的办法，就是使用 IndexSlice 对象
idx = pd.IndexSlice
health_data.loc[idx[:,1], idx[:,'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,14.0,39.0,53.0
2014,1,44.0,25.0,67.0


# 4-多级索引行列转换

01--有序的索引和无序的索引
    
     如果 MultiIndex 不是有序的索引，那么大多数切片操作都会失败

In [38]:
# 创建一个不按字典顺序排列的多级索引 Series
index = pd.MultiIndex.from_product([['a','c','b'],[1,2]])
data = pd.Series(np.random.rand(6),index=index)
data.index.names = ['char','int']
data

char  int
a     1      0.095008
      2      0.111820
c     1      0.723655
      2      0.957960
b     1      0.336702
      2      0.977194
dtype: float64

In [39]:
# 对索引使用局部切片，就会有错误出现
try:
    data['a':'b']
except KeyError as e:
    print(type(e))
    print(e)

<class 'pandas.errors.UnsortedIndexError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


局部切片和许多其他相似的操作都要求 MultiIndex 的各级索引是有序的

In [40]:
# Pandas 提供了许多便捷的操作完成排序，如 sort_index() 和 sortlevel() 方法。
data = data.sort_index()
data

char  int
a     1      0.095008
      2      0.111820
b     1      0.336702
      2      0.977194
c     1      0.723655
      2      0.957960
dtype: float64

In [41]:
# 索引排序之后，局部切片就可以正常使用了
data['a':'b']

char  int
a     1      0.095008
      2      0.111820
b     1      0.336702
      2      0.977194
dtype: float64

02--索引 stack 和 unstack

In [42]:
# 可以将一个多级索引数据集转换成简单的二维形式，
# 可以通过 level 参数设置转换的索引层级
pop.unstack(level=0)

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [43]:
pop.unstack(level=1)

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


03--索引的设置与重置

    层级数据维度转换的另一种方法是行列标签转换，通过 reset_index 方法实现。

In [44]:
pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


In [45]:
# 将类似这样的原始输入数据的列直接转换成 MultiIndex
# 通过 DataFrame 的 set_index 方法实现，返回结果就会是一个带多级索引的 DataFrame
pop_flat.set_index(['state','year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

# 5-多级索引的数据累计方法
    对于层级索引数据，可以设置参数 level 实现对数据子集的累计操作。

In [46]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      14.0  38.2  39.0  36.1  53.0  37.9
     2      46.0  34.5  36.0  38.8  47.0  38.1
2014 1      44.0  37.0  25.0  38.0  67.0  35.9
     2      44.0  37.7  26.0  37.7  52.0  36.6

In [47]:
# 计算每一年各项指标的平均值，将参数 level 设 置为索引 year
data_mean = health_data.mean(level='year')
data_mean

subject   Bob        Guido          Sue       
type       HR   Temp    HR   Temp    HR   Temp
year                                          
2013     30.0  36.35  37.5  37.45  50.0  38.00
2014     44.0  37.35  25.5  37.85  59.5  36.25

In [48]:
# 再设置 axis 参数，就可以对列索引进行类似的累计操作了
data_mean.mean(axis=1, level='type')

type,HR,Temp
year,,
2013,39.166667,37.266667
2014,43.000000,37.150000
